In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
my_data = spark.read.format('libsvm').load('logisticregressiondata/sample_libsvm_data.txt')

In [6]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
my_log_reg_model = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [8]:
fitted_logreg=  my_log_reg_model.fit(my_data)

In [9]:
log_summary = fitted_logreg.summary

In [10]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [11]:
log_summary.predictions.show(4)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[0.73765439548910...|[0.67648272431605...|       0.0|
|  1.0|(692,[158,159,160...|[-1.2286964884747...|[0.22640965216205...|       1.0|
|  1.0|(692,[124,125,126...|[-1.2596645795726...|[0.22103163838284...|       1.0|
|  1.0|(692,[152,153,154...|[-1.0845333752697...|[0.25264907653471...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 4 rows



In [12]:
lr_train,lr_test=my_data.randomSplit([0.7,0.3])

In [13]:
final_model = LogisticRegression()

In [14]:
fit_final= final_model.fit(lr_train)

In [15]:
predictions_and_labels = fit_final.evaluate(lr_test)

In [16]:
predictions_and_labels.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[10.0491527584149...|[0.99995677951476...|       0.0|
|  0.0|(692,[123,124,125...|[22.5116404201861...|[0.99999999983276...|       0.0|
|  0.0|(692,[125,126,127...|[21.1219622298693...|[0.99999999932880...|       0.0|
|  0.0|(692,[126,127,128...|[32.7706044033438...|[0.99999999999999...|       0.0|
|  0.0|(692,[129,130,131...|[16.7308412037323...|[0.99999994581389...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



### multinomial family for binary classification

In [17]:
mlr = LogisticRegression(maxIter=10,regParam=0.3,elasticNetParam=0.8,family="multinomial")
mlrmodel = mlr.fit(lr_train)
pred_mlrmodel=mlrmodel.evaluate(lr_test)
pred_mlrmodel.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[-0.0981170837006...|[0.45109828252169...|       1.0|
|  0.0|(692,[123,124,125...|[0.26507871812246...|[0.62951983072342...|       0.0|
|  0.0|(692,[125,126,127...|[0.35980502275355...|[0.67252114065069...|       0.0|
|  0.0|(692,[126,127,128...|[0.51364981230163...|[0.73639203691089...|       0.0|
|  0.0|(692,[129,130,131...|[0.39463938716916...|[0.68767644847283...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [18]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                  MulticlassClassificationEvaluator)

In [19]:
my_eval = BinaryClassificationEvaluator()

In [20]:
my_final_roc=my_eval.evaluate(predictions_and_labels.predictions)

In [21]:
my_final_roc1=my_eval.evaluate(pred_mlrmodel.predictions)

In [22]:
my_final_roc1

1.0

In [23]:
my_final_roc

1.0

In [24]:
my_final_roc.real

1.0